In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import json
import copy

if not os.path.exists(os.path.join(os.getcwd(), 'config.json')):
  # preventing to go always to the parent folder everytime we run the jupyter notebook without restarting
  os.chdir("..")
if not os.getcwd() in sys.path:
  sys.path.append(os.getcwd())

from analysis.analysis_commons import *
from strategies import *
from io import StringIO
import pandas as pd

# Main config
main_config = None
with open('config.json') as config_file:
  main_config = json.load(config_file)

def flatten_obj(obj, path=None):
    if path is None:
        path = []
    if isinstance(obj, dict):
        for k, v in obj.items():
            yield from flatten_obj(v, path + [k])
    else:
        yield path, obj

with open('results/delays-experiments-results.csv', 'w') as f:
  csv_writer = csv.writer(f)
  flattened_data = list(flatten_obj(outputs))
  csv_writer.writerow(['Topic', 'Disinformer', 'Mitigator', 'Delay', 'Costs'])
  for path, values in flattened_data:
      row = path + ['|'.join(map(str, values))]
      csv_writer.writerow(row)

#read
df = pd.read_csv('results/delays-experiments-results.csv')
outputs = {}
for i in range(len(df)):
  if df.at[i, 'Topic'] not in outputs:
    outputs[df.at[i, 'Topic']] = {}
  if df.at[i, 'Disinformer'] not in outputs[df.at[i, 'Topic']]:
    outputs[df.at[i, 'Topic']][df.at[i, 'Disinformer']] = {}
  if df.at[i, 'Mitigator'] not in outputs[df.at[i, 'Topic']][df.at[i, 'Disinformer']]:
    outputs[df.at[i, 'Topic']][df.at[i, 'Disinformer']][df.at[i, 'Mitigator']] = {}
  if df.at[i, 'Delay'] not in outputs[df.at[i, 'Topic']][df.at[i, 'Disinformer']][df.at[i, 'Mitigator']]:
    outputs[df.at[i, 'Topic']][df.at[i, 'Disinformer']][df.at[i, 'Mitigator']][df.at[i, 'Delay']] = []
  outputs[df.at[i, 'Topic']][df.at[i, 'Disinformer']][df.at[i, 'Mitigator']][df.at[i, 'Delay']] = list(map(float, df.at[i, 'Costs'].split('|')))
  colors = {
    'RandomStrategy': 'tab:orange',
    'GreedyStrategy': 'tab:blue',
    'MultiObjectiveStrategy': 'tab:green',
    'OptimalCostsStrategy': 'tab:brown',
    'OptimalCostsStrategyLimited': 'tab:brown'
}

def plot(topic, outputs):
  data = outputs[topic]
  num_rows = len(data)

  fig, axes = plt.subplots(nrows=num_rows, ncols=1, figsize=(10, 5 * num_rows))

  if not isinstance(axes, np.ndarray):
      axes = [axes]

  # Plotting each strategy in its respective subplot
  for ax, (strategy, values) in zip(axes, data.items()):
      added_labels = set()  # Keep track of labels already added to the legend

      for sub_strategy, points in values.items():
          x = sorted([k for k in points.keys()])
          y = []
          for k in x:
            costs = list(filter(lambda v: v > 0, points[k])) # we exclude -1 values
            if costs:
              y.append(sum(costs) / len(costs))
            else:
              y.append(None)

          # Handling None values for transparency
          x_values, y_values = [], []
          for i in range(len(x)):
              if y[i] is not None:
                  x_values.append(x[i])
                  y_values.append(y[i])
                  if i == len(x) - 1 or y[i+1] is None:
                      if sub_strategy not in added_labels:
                          ax.plot(x_values, y_values, label=sub_strategy, color=colors[sub_strategy])
                          added_labels.add(sub_strategy)
                      else:
                          ax.plot(x_values, y_values, color=colors[sub_strategy])
                      x_values, y_values = [], []
              else:
                  if x_values:
                      if sub_strategy not in added_labels:
                          ax.plot(x_values, y_values, label=sub_strategy, color=colors[sub_strategy])
                          added_labels.add(sub_strategy)
                      else:
                          ax.plot(x_values, y_values, color=colors[sub_strategy])
                      x_values, y_values = [], []

      ax.set_title(topic + ' (' + strategy + ' baseline)')
      ax.set_xlabel('Turns')
      ax.set_ylabel('Cost')
      ax.legend()

  plt.tight_layout()
  plt.show()

for topic in outputs.keys():
  plot(topic, outputs)